In [3]:
from gensim.summarization import summarize
import fse # fast sentence embeddings

from gensim.models import KeyedVectors

path_w2v = '/home/santosh/Work/models/word2vec/CORD-19/CORD-19-FT_200d_5w_10i_5mc_sentence.model'

covid_trained_model = KeyedVectors.load(path_w2v)

2020-03-31 20:09:32,949 : INFO : loading Word2VecKeyedVectors object from /home/santosh/Work/models/word2vec/CORD-19/CORD-19-FT_200d_5w_10i_5mc_sentence.model
2020-03-31 20:09:33,327 : INFO : loading wv recursively from /home/santosh/Work/models/word2vec/CORD-19/CORD-19-FT_200d_5w_10i_5mc_sentence.model.wv.* with mmap=None
2020-03-31 20:09:33,328 : INFO : loading vectors from /home/santosh/Work/models/word2vec/CORD-19/CORD-19-FT_200d_5w_10i_5mc_sentence.model.wv.vectors.npy with mmap=None
2020-03-31 20:09:35,334 : INFO : loading vectors_vocab from /home/santosh/Work/models/word2vec/CORD-19/CORD-19-FT_200d_5w_10i_5mc_sentence.model.wv.vectors_vocab.npy with mmap=None
2020-03-31 20:09:37,354 : INFO : loading vectors_ngrams from /home/santosh/Work/models/word2vec/CORD-19/CORD-19-FT_200d_5w_10i_5mc_sentence.model.wv.vectors_ngrams.npy with mmap=None
2020-03-31 20:09:45,833 : INFO : setting ignored attribute vectors_norm to None
2020-03-31 20:09:45,834 : INFO : setting ignored attribute vec

In [4]:
# Here I am using word embeddings from the CORD-19
import glob
import nltk
import logging
import os
from tqdm import tqdm


from nltk.corpus import stopwords
from nltk.tokenize.treebank import TreebankWordDetokenizer, TreebankWordTokenizer

# start the log
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',level=logging.INFO)


def clean_my_text(full_text_):
    stopset = set(stopwords.words('english')) #| set(string.punctuation)
    tokens = TreebankWordTokenizer().tokenize(full_text_.split('---10')[0])
    cleanup = [token for token in tokens if token not in stopset and len(token) > 1]
    return cleanup


# get the current working directory and file
dir_path = '/home/santosh/Work/Datasets/COVID-CORD-19_paragraphs/'#'/home/santosh/Work/Datasets/CORD-19-sentences/'




In [5]:
paragraphs = []

all_text_files = sorted(glob.glob(dir_path + '*.txt*'))

for each_text_file in tqdm(all_text_files):
    with open(each_text_file, 'r') as f:
        temp_ = f.readlines()
        for each_line in temp_:
            paragraphs.append(TreebankWordTokenizer().tokenize(each_line))
  

100%|██████████| 4427/4427 [01:34<00:00, 46.89it/s]


In [6]:
import pickle

def dump(data,filename):
    file = open(result_dir_path+filename+'.bin','wb')
    pickle.dump(data, file)
    file.close()
    
def load(filename):
    file = open(result_dir_path+filename+'.bin','rb')
    data = pickle.load(file)
    file.close()
    return data    

In [7]:
result_dir_path = '/home/santosh/Work/models/word2vec/CORD-19/'
from fse import IndexedList

paragraphs_index = IndexedList(paragraphs)
dump(paragraphs_index, 'paragraphs_index_covid-19')

In [8]:
result_dir_path = '/home/santosh/Work/models/word2vec/CORD-19/'
paragraphs_index = load('paragraphs_index_covid-19')


In [8]:
# SIF embeddings
from fse.models import uSIF
sif_model = uSIF(covid_trained_model, workers=6, lang_freq="en")

sif_model.train(paragraphs_index)

2020-03-31 20:12:14,274 : INFO : no frequency mode: using wordfreq for estimation of frequency for language: en
2020-03-31 20:12:14,912 : INFO : scanning all indexed sentences and their word counts
2020-03-31 20:12:15,121 : INFO : finished scanning 181169 sentences with an average length of 142 and 25750678 total words
2020-03-31 20:12:15,149 : INFO : estimated memory for 181169 sentences with 200 dimensions and 165124 vocabulary: 1916 MB (1 GB)
2020-03-31 20:12:15,150 : INFO : initializing sentence vectors for 181169 sentences
2020-03-31 20:12:15,761 : INFO : pre-computing uSIF weights for 165124 words
2020-03-31 20:12:16,111 : INFO : begin training
2020-03-31 20:12:21,204 : INFO : PROGRESS : finished 14.00% with 25362 sentences and 3500770 words, 5072 sentences/s
2020-03-31 20:12:26,206 : INFO : PROGRESS : finished 27.38% with 49598 sentences and 7002268 words, 4847 sentences/s
2020-03-31 20:12:31,306 : INFO : PROGRESS : finished 41.01% with 74293 sentences and 10527974 words, 4939 s

(181169, 25750678)

In [9]:
sif_model.save(result_dir_path+'CORD-19-FT_covid_papers_p2v_sif.model')

2020-03-31 20:13:20,934 : INFO : saving uSIF object under /home/santosh/Work/models/word2vec/CORD-19/CORD-19-FT_covid_papers_p2v_sif.model, separately None
2020-03-31 20:13:20,935 : INFO : storing np array 'vectors' to /home/santosh/Work/models/word2vec/CORD-19/CORD-19-FT_covid_papers_p2v_sif.model.wv.vectors.npy
2020-03-31 20:13:21,003 : INFO : storing np array 'vectors_vocab' to /home/santosh/Work/models/word2vec/CORD-19/CORD-19-FT_covid_papers_p2v_sif.model.wv.vectors_vocab.npy
2020-03-31 20:13:21,066 : INFO : storing np array 'vectors_ngrams' to /home/santosh/Work/models/word2vec/CORD-19/CORD-19-FT_covid_papers_p2v_sif.model.wv.vectors_ngrams.npy
2020-03-31 20:13:56,493 : INFO : storing np array 'vectors' to /home/santosh/Work/models/word2vec/CORD-19/CORD-19-FT_covid_papers_p2v_sif.model.sv.vectors.npy
2020-03-31 20:14:01,389 : INFO : saved /home/santosh/Work/models/word2vec/CORD-19/CORD-19-FT_covid_papers_p2v_sif.model


# Test Embeddings

In [10]:
from gensim.summarization import summarize

In [53]:
from gensim.models import KeyedVectors
sif_model = KeyedVectors.load(result_dir_path+'CORD-19_sif_sv.model')# 'CORD-19-FT_covid_papers_p2v_sif.model')

2020-03-31 20:48:57,917 : INFO : loading Word2VecKeyedVectors object from /home/santosh/Work/models/word2vec/CORD-19/CORD-19_sif_sv.model
2020-03-31 20:48:59,373 : INFO : loading wv recursively from /home/santosh/Work/models/word2vec/CORD-19/CORD-19_sif_sv.model.wv.* with mmap=None
2020-03-31 20:48:59,374 : INFO : loading vectors from /home/santosh/Work/models/word2vec/CORD-19/CORD-19_sif_sv.model.wv.vectors.npy with mmap=None
2020-03-31 20:49:02,913 : INFO : loading sv recursively from /home/santosh/Work/models/word2vec/CORD-19/CORD-19_sif_sv.model.sv.* with mmap=None
2020-03-31 20:49:02,915 : INFO : loading vectors from /home/santosh/Work/models/word2vec/CORD-19/CORD-19_sif_sv.model.sv.vectors.npy with mmap=None
2020-03-31 20:49:37,617 : INFO : loading prep recursively from /home/santosh/Work/models/word2vec/CORD-19/CORD-19_sif_sv.model.prep.* with mmap=None
2020-03-31 20:49:37,618 : INFO : loaded /home/santosh/Work/models/word2vec/CORD-19/CORD-19_sif_sv.model


In [54]:
import re
def extract_query_result(sv_query_result):
    result_sentences =[]
    for each_result in sv_query_result:
        result_sentences.append(TreebankWordDetokenizer().detokenize((each_result[0])))
     
    return result_sentences

In [12]:
# !pip install transformers==2.2.0
# !pip install bert-extractive-summarizer

In [13]:
# from summarizer import Summarizer

In [14]:
# model = Summarizer()
# body = ' '.join(extract_query_result(query_result))
# result = model(body, min_length=200)
# full = ''.join(result)
# full

In [42]:
from IPython.core.display import display, HTML

def convert_html(list_sentences):
    html_markup_list = []
    for each_result in list_sentences:
        sentence = '<p> '+each_result.split('---10')[0]+'</p>'
        source = '<a href = "https://doi.org/'+each_result.split('---')[1].replace('---','').strip()+'">[source]</a>'
        
        html_markup_list.append(sentence+' '+source)
        
    return html_markup_list
    

In [ ]:
# query = "pre-existing pulmonary disease SARS-Cov2 Hypertension" 

# query = "What is the incubation days of SARS-CoV-2" 
# query = "incubation days coronavirus 2019-nCoV"#  COVID-19
# query = 'socio economic poverty behaviour'

# query = 'what is the comorbidities associated with death'
# query = 'public health mitigation measures that could be effective for control'
# query = 'socio-economic and behavioral factors to understand the economic impact of the SARS-CoV-2 virus and whether there were differences. '
# query = 'what are the risk factors for death in COVID-19'
# query = 'what is the basic reproductive number of SARS-CoV-2 in days'
# query = 'what is the serial interval days SARS-CoV-2'
# query = 'what do we know about the environmental factors influencing SARS-CoV-2'
# query = 'what do we know about medication COVID-19'
query = 'Transmission dynamics of the virus SARS-CoV-2'
# query ='risk of fatality among symptomatic hospitalized patients in SARS-CoV-2'
# query = 'Efforts targeted at a universal coronavirus vaccine'


query_result = sif_model.sv.similar_by_sentence(clean_my_text(query), model=sif_model, indexable=paragraphs_index.items, topn=10)

extract_query_result(query_result)
# summarize(' '.join(extract_query_result(query_result)), ratio =0.2, split=True)

display(HTML('<hr>\n'.join(convert_html(extract_query_result(query_result)))))


2020-03-31 20:49:45,039 : INFO : scanning all indexed sentences and their word counts
2020-03-31 20:49:45,041 : INFO : finished scanning 1 sentences with an average length of 4 and 4 total words
2020-03-31 20:49:45,044 : INFO : removing 5 principal components took 0s
2020-03-31 20:49:45,045 : INFO : precomputing L2-norms of sentence vectors
